## 모델 정의

In [1]:
import math
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

In [2]:
TRAIN_DATA_DIR = "data/train/"
VAL_DATA_DIR = "data/val/"
TRAIN_SAMPLES = 500
VAL_SAMPLES = 500
NUM_CLASSES = 2
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 64

In [3]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [4]:
train_generator = train_datagen.flow_from_directory(
    TRAIN_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode="categorical"
)

Found 500 images belonging to 2 classes.


In [5]:
val_generator = val_datagen.flow_from_directory(
    VAL_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=False,
    class_mode="categorical"
)

Found 500 images belonging to 2 classes.


In [6]:
def model_maker():
    base_model = MobileNetV2(include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))

    for layer in base_model.layers:
        layer.trainable = False
    
    input = keras.layers.Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = keras.layers.GlobalAveragePooling2D()(custom_model)
    custom_model = keras.layers.Dense(64, activation="relu")(custom_model)
    custom_model = keras.layers.Dropout(0.5)(custom_model)
    predictions = keras.layers.Dense(NUM_CLASSES, activation="softmax")(custom_model)
    return keras.models.Model(inputs=input, outputs=predictions)

In [7]:
model = model_maker()
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(lr=1e-3), metrics=["acc"])
num_steps = math.ceil(float(TRAIN_SAMPLES) / BATCH_SIZE)

The following Variables were used a Lambda layer's call (tf.nn.convolution), but
are not present in its tracked objects:
  <tf.Variable 'Conv1/kernel:0' shape=(3, 3, 3, 32) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.compat.v1.nn.fused_batch_norm), but
are not present in its tracked objects:
  <tf.Variable 'bn_Conv1/gamma:0' shape=(32,) dtype=float32>
  <tf.Variable 'bn_Conv1/beta:0' shape=(32,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.compat.v1.nn.depthwise_conv2d), but
are not present in its tracked objects:
  <tf.Variable 'expanded_

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


## 모델 훈련

In [8]:
history = model.fit_generator(
    generator=train_generator,
    steps_per_epoch=num_steps,
    epochs=10,
    validation_data=val_generator,
    validation_steps=num_steps
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
8/8 [==============================] - 36s 4s/step - loss: 0.6198 - acc: 0.6840 - val_loss: 0.2866 - val_acc: 0.9020
Epoch 2/10
8/8 [==============================] - 32s 4s/step - loss: 0.2362 - acc: 0.9180 - val_loss: 0.0917 - val_acc: 0.9760
Epoch 3/10
8/8 [==============================] - 39s 5s/step - loss: 0.1224 - acc: 0.9600 - val_loss: 0.0586 - val_acc: 0.9820
Epoch 4/10
8/8 [==============================] - 32s 4s/step - loss: 0.0934 - acc: 0.9660 - val_loss: 0.0473 - val_acc: 0.9820
Epoch 5/10
8/8 [==============================] - 32s 4s/step - loss: 0.0800 - acc: 0.9720 - val_loss: 0.0467 - val_acc: 0.9820
Epoch 6/10
8/8 [==============================] - 32s 4s/step - loss: 0.0802 - acc: 0.9700 - val_loss: 0.0518 - val_acc: 0.9820
Epoch 7/10
8/8 [==============================] - 32s 4s/step - loss: 0.0702 - acc: 0.9660 - val_loss: 0.0485 - val_acc: 0.9820
Epoch 8/10
8/8 [==============================] - 32s 4s/step - loss: 0.0546 - acc: 0.9820 - val_loss: 0

## 샘플 이미지 로딩 후 결과 보기

In [9]:
def preprocess_img(img_path):
    img = keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img_array = keras.preprocessing.image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = expanded_img_array / 255.0
    return preprocessed_img

In [10]:
dog_img = preprocess_img("data/sample-images/dog.jpg")
model.predict(dog_img)

array([[2.4048072e-04, 9.9975950e-01]], dtype=float32)

In [11]:
cat_img = preprocess_img("data/sample-images/cat.jpg")
model.predict(cat_img)

array([[9.999374e-01, 6.262295e-05]], dtype=float32)